In [3]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Energy Consumption Analysis Package - Usage Examples\n",
    "\n",
    "This notebook demonstrates how to use the `energy_analysis` package for analyzing and forecasting energy consumption data from Excel files.\n",
    "\n",
    "## Setup and Data Loading"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "from energy_analysis import DataProcessor, EnergyAnalyzer, EnergyForecaster, EnergyVisualizer\n",
    "\n",
    "# Initialize our package components\n",
    "processor = DataProcessor()\n",
    "analyzer = EnergyAnalyzer()\n",
    "forecaster = EnergyForecaster()\n",
    "visualizer = EnergyVisualizer()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Loading Data from Excel File\n",
    "\n",
    "Our dataset is in Excel format with the following columns:\n",
    "- date: Date of measurement\n",
    "- organizationName: Organization identifier\n",
    "- providerID: Provider identifier\n",
    "- quantity: Electricity consumption in kWh\n",
    "- unitName: Unit of measurement"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Load data from Excel file\n",
    "file_path = 'energy_data.xlsx'\n",
    "\n",
    "# Read Excel file\n",
    "raw_data = pd.read_excel(\n",
    "    file_path,\n",
    "    usecols=['date', 'organizationName', 'providerID', 'quantity', 'unitName']\n",
    ")\n",
    "\n",
    "# Basic data preparation\n",
    "raw_data['date'] = pd.to_datetime(raw_data['date'])\n",
    "raw_data['quantity'] = raw_data['quantity'].astype(float)\n",
    "\n",
    "print(\"Data sample:\")\n",
    "display(raw_data.head())\n",
    "\n",
    "print(\"\\nDataset Info:\")\n",
    "print(raw_data.info())\n",
    "\n",
    "print(\"\\nUnique organizations:\", raw_data['organizationName'].nunique())\n",
    "print(\"Date range:\", raw_data['date'].min(), \"to\", raw_data['date'].max())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Data Quality Check\n",
    "\n",
    "Let's check the data quality before proceeding with analysis."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Check for missing values\n",
    "print(\"Missing values in each column:\")\n",
    "print(raw_data.isnull().sum())\n",
    "\n",
    "# Check for negative or zero consumption values\n",
    "invalid_consumption = raw_data[raw_data['quantity'] <= 0]\n",
    "print(f\"\\nNumber of invalid consumption values: {len(invalid_consumption)}\")\n",
    "\n",
    "# Display basic statistics of consumption\n",
    "print(\"\\nConsumption Statistics:\")\n",
    "print(raw_data['quantity'].describe())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Data Processing\n",
    "\n",
    "Let's clean and prepare our data using the DataProcessor class."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Clean the data\n",
    "clean_data = processor.clean_data(raw_data)\n",
    "\n",
    "# Group data by organization and date\n",
    "daily_consumption = clean_data.groupby(\n",
    "    ['date', 'organizationName']\n",
    ")['quantity'].sum().reset_index()\n",
    "\n",
    "print(\"Daily consumption by organization:\")\n",
    "display(daily_consumption.head())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Statistical Analysis\n",
    "\n",
    "Now let's analyze consumption patterns for each organization."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Calculate statistics for each organization\n",
    "org_stats = {}\n",
    "for org in clean_data['organizationName'].unique():\n",
    "    org_data = clean_data[clean_data['organizationName'] == org]\n",
    "    org_stats[org] = analyzer.calculate_basic_stats(org_data, consumption_column='quantity')\n",
    "\n",
    "# Display statistics for each organization\n",
    "for org, stats in org_stats.items():\n",
    "    print(f\"\\nStatistics for {org}:\")\n",
    "    for metric, value in stats.items():\n",
    "        print(f\"{metric.capitalize()}: {value:.2f} kWh\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Visualization\n",
    "\n",
    "Let's create visualizations to understand consumption patterns."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Plot consumption trends for each organization\n",
    "for org in clean_data['organizationName'].unique():\n",
    "    org_data = clean_data[clean_data['organizationName'] == org]\n",
    "    \n",
    "    visualizer.plot_consumption_trend(\n",
    "        data=org_data,\n",
    "        date_column='date',\n",
    "        consumption_column='quantity',\n",
    "        figsize=(12, 6)\n",
    "    )\n",
    "    plt.title(f'Energy Consumption Pattern - {org}')\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Pattern Detection\n",
    "\n",
    "Let's analyze consumption patterns for different time periods."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Detect patterns for each organization\n",
    "for org in clean_data['organizationName'].unique():\n",
    "    org_data = clean_data[clean_data['organizationName'] == org]\n",
    "    \n",
    "    patterns = analyzer.detect_patterns(\n",
    "        data=org_data,\n",
    "        consumption_column='quantity',\n",
    "        date_column='date'\n",
    "    )\n",
    "    \n",
    "    print(f\"\\nPatterns for {org}:\")\n",
    "    \n",
    "    # Plot daily patterns\n",
    "    visualizer.plot_patterns(patterns, pattern_type='daily')\n",
    "    plt.title(f'Daily Consumption Pattern - {org}')\n",
    "    plt.show()\n",
    "    \n",
    "    # Print pattern statistics\n",
    "    print(f\"Daily pattern statistics:\")\n",
    "    print(patterns['daily_pattern'].describe())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Forecasting\n",
    "\n",
    "Let's forecast future consumption for each organization."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Forecast for each organization\n",
    "for org in clean_data['organizationName'].unique():\n",
    "    org_data = clean_data[clean_data['organizationName'] == org]\n",
    "    \n",
    "    # Prepare data for forecasting\n",
    "    train_data, test_data = forecaster.prepare_data(\n",
    "        org_data,\n",
    "        target_column='quantity',\n",
    "        test_size=0.2\n",
    "    )\n",
    "    \n",
    "    # Generate forecast\n",
    "    forecast = forecaster.forecast_consumption(\n",
    "        data=train_data['quantity'],\n",
    "        periods=7,  # Forecast next 7 days\n",
    "        method='holt-winters',\n",
    "        seasonal_periods=24\n",
    "    )\n",
    "    \n",
    "    print(f\"\\nForecast for {org}:\")\n",
    "    print(\"Next 7 days consumption forecast:\")\n",
    "    print(forecast)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Export Results\n",
    "\n",
    "Let's save our analysis results back to Excel."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create a summary DataFrame with all organizations\n",
    "summary_data = []\n",
    "for org in clean_data['organizationName'].unique():\n",
    "    org_data = clean_data[clean_data['organizationName'] == org]\n",
    "    stats = analyzer.calculate_basic_stats(org_data, consumption_column='quantity')\n",
    "    \n",
    "    summary_data.append({\n",
    "        'Organization': org,\n",
    "        'Average Consumption': stats['mean'],\n",
    "        'Peak Consumption': stats['max'],\n",
    "        'Minimum Consumption': stats['min'],\n",
    "        'Standard Deviation': stats['std']\n",
    "    })\n",
    "\n",
    "summary_df = pd.DataFrame(summary_data)\n",
    "\n",
    "# Export to Excel\n",
    "with pd.ExcelWriter('energy_analysis_results.xlsx') as writer:\n",
    "    summary_df.to_excel(writer, sheet_name='Summary', index=False)\n",
    "    daily_consumption.to_excel(writer, sheet_name='Daily Consumption', index=False)\n",
    "\n",
    "print(\"Analysis results have been exported to 'energy_analysis_results.xlsx'\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusion\n",
    "\n",
    "This notebook demonstrated how to:\n",
    "1. Load and process data from Excel files\n",
    "2. Analyze consumption patterns for multiple organizations\n",
    "3. Create visualizations of consumption trends\n",
    "4. Generate forecasts for future consumption\n",
    "5. Export analysis results back to Excel\n",
    "\n",
    "The package provides a flexible framework for analyzing energy consumption data from various organizations and can be customized for specific analysis needs."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 }
}

NameError: name 'null' is not defined